In [0]:
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

--2018-11-24 05:36:47--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.222, 91.189.89.223
Connecting to launchpad.net (launchpad.net)|91.189.89.222|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://launchpadlibrarian.net/386846978/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb [following]
--2018-11-24 05:36:48--  https://launchpadlibrarian.net/386846978/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpadlibrarian.net (launchpadlibrarian.net)... 91.189.89.228, 91.189.89.229
Connecting to launchpadlibrarian.net (launchpadlibrarian.net)|91.189.89.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1232624 (1.2M) [application/x-debian-package]
Saving to: ‘google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb’

google-drive-ocamlf 100%[==================

In [0]:
!mkdir -p drive

!google-drive-ocamlfuse drive
!pip install tables
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools

    100% |████████████████████████████████| 3.8MB 10.4MB/s 
    100% |████████████████████████████████| 163kB 31.4MB/s 
E: Package 'python-software-properties' has no installation candidate


In [0]:
import pandas as pd
import numpy as np
from os.path import join, exists, expanduser
from tqdm import tqdm
from sklearn.metrics import log_loss, accuracy_score

from keras.preprocessing import image
from keras import applications


In [0]:
df=pd.read_csv('drive/Colab Notebooks/labels.csv')

In [0]:
uni_col=np.unique(df.iloc[:,1])
#print(uni_col)

In [0]:
df.groupby("breed").count().sort_values("id", ascending=False)

,id
breed,
scottish_deerhound,126
maltese_dog,117
afghan_hound,116
entlebucher,115
bernese_mountain_dog,114
shih-tzu,112
great_pyrenees,111
pomeranian,111
basenji,110


In [0]:
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder()
SEED = 2018
NUM_CLASSES = 120

np.random.seed(seed=SEED)
rnd = np.random.random(len(df))
train_idx = rnd < 0.9
valid_idx = rnd >= 0.9
y_train = label_enc.fit_transform(df["breed"].values)
ytr = y_train[train_idx]
yv = y_train[valid_idx]

In [0]:
data_dir='drive/Colab Notebooks/'
def read_img(img_id, train_or_test, size):
    img = image.load_img(join(data_dir, train_or_test, '{}.jpg'.format(img_id)), target_size=size)
    return image.img_to_array(img)

In [0]:
INPUT_SIZE = 224
BATCH_SIZE = 16

x_train = np.zeros((train_idx.sum(), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
x_valid = np.zeros((valid_idx.sum(), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
train_i = 0
valid_i = 0
for i, img_id in tqdm(enumerate(df['id'])):
    img = read_img(img_id, 'train', (INPUT_SIZE, INPUT_SIZE))
    x = applications.resnet50.preprocess_input(np.expand_dims(img.copy(), axis=0))
    if train_idx[i]:
        x_train[train_i] = x
        train_i += 1
    elif valid_idx[i]:
        x_valid[valid_i] = x
        valid_i += 1
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))

10222it [02:15, 75.46it/s]

Train Images shape: (9174, 224, 224, 3) size: 1,380,943,872


In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator()

In [0]:
from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop
from keras.models import Model, Input

In [0]:
# create the base pre-trained model
base_model = applications.ResNet50(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have NUM_CLASSES classes
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 15s 0us/step


In [0]:
import datetime
from keras.callbacks import EarlyStopping


early_stopping = EarlyStopping(monitor='val_acc', patience=5)
STAMP = "{}_dog_breed_model".format(datetime.date.today().strftime("%Y-%m-%d"))


In [0]:
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
optimizer = RMSprop(lr=0.001, rho=0.9)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

In [0]:
hist = model.fit_generator(train_datagen.flow(x_train, ytr, batch_size=BATCH_SIZE),
                           steps_per_epoch=train_idx.sum() // BATCH_SIZE,
                           epochs=50, callbacks=[early_stopping],
                           validation_data=test_datagen.flow(x_valid, yv, batch_size=BATCH_SIZE),
                           validation_steps=valid_idx.sum() // BATCH_SIZE)

Epoch 1/50
573/573 [==============================] - 159s 277ms/step - loss: 1.0180 - acc: 0.7162 - val_loss: 1.2928 - val_acc: 0.7269
Epoch 2/50
573/573 [==============================] - 157s 274ms/step - loss: 0.9779 - acc: 0.7232 - val_loss: 1.2676 - val_acc: 0.7141
Epoch 3/50
573/573 [==============================] - 157s 274ms/step - loss: 0.9635 - acc: 0.7272 - val_loss: 1.2576 - val_acc: 0.7384
Epoch 4/50
573/573 [==============================] - 157s 274ms/step - loss: 0.9477 - acc: 0.7296 - val_loss: 1.3390 - val_acc: 0.7258
Epoch 5/50
573/573 [==============================] - 157s 275ms/step - loss: 0.9445 - acc: 0.7331 - val_loss: 1.2707 - val_acc: 0.7238
Epoch 6/50
573/573 [==============================] - 157s 275ms/step - loss: 0.9225 - acc: 0.7429 - val_loss: 1.1872 - val_acc: 0.7422
Epoch 7/50
573/573 [==============================] - 158s 275ms/step - loss: 0.9313 - acc: 0.7376 - val_loss: 1.2567 - val_acc: 0.7345
Epoch 8/50
573/573 [============================

In [0]:
bst_val_acc = max(hist.history['val_acc'])
print("Best val acc: {:.1%}".format(bst_val_acc))

Best val acc: 74.8%


In [0]:
for layer in model.layers[:116]:
    layer.trainable = False
for layer in model.layers[116:]:
    layer.trainable = True

In [0]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

In [0]:
hist = model.fit_generator(train_datagen.flow(x_train, ytr, batch_size=BATCH_SIZE),
                           steps_per_epoch=train_idx.sum() // BATCH_SIZE,
                           initial_epoch=len(hist.history["val_acc"]),
                           epochs=len(hist.history["val_acc"]) + 1,
                           validation_data=test_datagen.flow(x_valid, yv, batch_size=BATCH_SIZE),
                           validation_steps=valid_idx.sum() // BATCH_SIZE)

Epoch 14/14
573/573 [==============================] - 162s 283ms/step - loss: 0.8293 - acc: 0.7631 - val_loss: 1.2865 - val_acc: 0.7481


In [0]:
bst_val_acc = max(hist.history['val_acc'])
print("Best val acc: {:.1%}".format(bst_val_acc))

model.save_weights("{}_{}_fine-tuned".format(STAMP, bst_val_acc))

Best val acc: 74.8%
